# Notebook Aim
To investigate .ttl files and parse them

In [1]:
# https://github.com/RDFLib/rdflib
#!pip install rdflib
from rdflib import Graph

## Ontology File
The RDf data dump comes with an ontology file that can use to know which RDF types to expect in the `.ttl` files.

In [2]:
from rdflib.extras.external_graph_libs import rdflib_to_networkx_multidigraph
# import networkx as nx
# import matplotlib.pyplot as plt

url = 'https://www.w3.org/TeamSubmission/turtle/tests/test-30.ttl'

onto_graph = Graph()
result = onto_graph.parse("./wpOntology.ttl", format='turtle')

# G = rdflib_to_networkx_multidigraph(result)

# # Plot Networkx instance of RDF Graph
# pos = nx.spring_layout(G, scale=2)
# plt.figure(figsize=(50,50))
# edge_labels = nx.get_edge_attributes(G, 'r')
# nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)
# nx.draw(G, with_labels=True)

# #if not in interactive mode for 

# plt.savefig("ontology_graph.png")

## Test WP File - WP111

In [3]:
g = Graph()

# Parse an RDF file
wp111g = g.parse("./WP111.ttl")

In [4]:
len(wp111g)  # Number of subj/pred/obj

2353

In [5]:
# Getting labels of GeneProducts
q = """
    PREFIX dc: <http://purl.org/dc/elements/1.1/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT ?label
    WHERE {
        ?p rdf:type wp:GeneProduct .

        ?p rdfs:label ?label .
    }
"""
for r in wp111g.query(q):
    print(r["label"])
    break

NDUFS3


## Parse PubRefs

In [13]:
pubref_query = """
SELECT ?id ?data_source ?data_source_id ?link
    WHERE {
        ?id rdf:type wp:PublicationReference ;
            rdf:type ?type .
            ?id foaf:page ?link .
            ?id dc:source ?data_source .
            ?id dcterms:identifier ?data_source_id .
    }
"""

for match in wp111g.query(pubref_query):
    result_dict = {key: str(val) for key, val in match.asdict().items()}
    print(result_dict)

{'id': 'https://identifiers.org/pubmed/23746447', 'datasource': 'PubMed', 'datasource_id': '23746447', 'link': 'http://www.ncbi.nlm.nih.gov/pubmed/23746447'}
{'id': 'https://identifiers.org/pubmed/22902835', 'datasource': 'PubMed', 'datasource_id': '22902835', 'link': 'http://www.ncbi.nlm.nih.gov/pubmed/22902835'}
{'id': 'https://identifiers.org/pubmed/30030361', 'datasource': 'PubMed', 'datasource_id': '30030361', 'link': 'http://www.ncbi.nlm.nih.gov/pubmed/30030361'}


### Parse Pathways

In [27]:
pathway_query = """
SELECT ?id ?data_source ?data_source_id ?description ?organism ?label ?type
    WHERE {
        ?id rdf:type wp:Pathway .
        ?id rdf:type ?type .
        ?id dc:source ?data_source .
        ?id dcterms:identifier ?data_source_id .
        OPTIONAL {
            ?id wp:organismName ?organism .
            ?id dcterms:description ?description .
        }
    FILTER (?type = "wp:Pathway")
    }
"""

for match in wp111g.query(pathway_query):
    result_dict = {key: str(val) for key, val in match.asdict().items()}
    print(result_dict)

In [35]:
pathway_query = """
SELECT ?id ?data_source ?data_source_id ?type
    WHERE {
        ?id rdf:type wp:Pathway .
        ?id rdf:type ?type .
        ?id dc:source ?data_source .
        ?id dcterms:identifier ?data_source_id .
    FILTER (?type = "http://vocabularies.wikipathways.org/wp#Pathway")
    }
"""

for match in wp111g.query(pathway_query):
    result_dict = {key: str(val) for key, val in match.asdict().items()}
    print(result_dict)

### Parse Nodes

In [6]:
def parse_nodes(wikipathway, query: str) -> dict:
    """Parse nodes in the wikipathway graph."""
    nodes = dict()
    for match in wikipathway.query(query):
        result_dict = {key: str(val) for key, val in match.asdict().items()}
        node_types = {int_type.split("#")[1] for int_type in result_dict.pop("types").split(",")}

        # don't need "DataNode" since it's a duplicate
        node_types.remove('DataNode')

        # Defensive code
        if len(node_types) != 1:  # Should only be one node type
            raise ValueError(f"Too many node types: {node_types}")

        # Parse participants of wp:Complex nodes
        if "Complex" in node_types:
            result_dict["participants"] = set(result_dict["participants"].split(","))

        result_dict["type"] = list(node_types)[0]

        ttl_id = result_dict.pop("ttl_id")
        nodes[ttl_id] = result_dict

    return nodes

noncomplex_node_query = """
    SELECT ?ttl_id (GROUP_CONCAT(?type;SEPARATOR=",") AS ?types) ?label ?id_source ?identifier
    WHERE {
        ?ttl_id rdf:type wp:DataNode ;
            rdf:type ?type .
            ?ttl_id rdfs:label ?label .
            ?ttl_id dc:source ?id_source .
            ?ttl_id dcterms:identifier ?identifier .
    }
    GROUP BY ?ttl_id
"""

complex_node_query = """
    SELECT ?ttl_id (GROUP_CONCAT(?type;SEPARATOR=",") AS ?types) (GROUP_CONCAT(?ps ;SEPARATOR=",") AS ?participants)
    WHERE {
        ?ttl_id rdf:type wp:Complex ;
            rdf:type ?type ;
            wp:participants ?ps .
    }
    GROUP BY ?ttl_id
"""

noncomplex_nodes = parse_nodes(wikipathway=wp111g, query=noncomplex_node_query)
complex_nodes = parse_nodes(wikipathway=wp111g, query=complex_node_query)
nodes = {**noncomplex_nodes, **complex_nodes}

In [7]:
for key, value in nodes.items():
    print(key)
    print(value)
    break

https://identifiers.org/ncbigene/4722
{'label': 'NDUFS3', 'id_source': 'Entrez Gene', 'identifier': '4722', 'type': 'GeneProduct'}


### Parse Interactions

In [8]:
# Get and parse edges
# ";" can be used when same subj/pred/obj used - https://www.stardog.com/tutorials/sparql#ordering-results
int_q = """
    SELECT ?ttl_id (GROUP_CONCAT(?type;SEPARATOR=",") AS ?types) ?source ?target ?identifier (GROUP_CONCAT(?ps ;SEPARATOR=",") AS ?participants)
    WHERE {
        ?ttl_id rdf:type wp:Interaction ;
            wp:participants ?ps ;
            rdf:type ?type .
        OPTIONAL {
            ?ttl_id wp:source ?source .
            ?ttl_id wp:target ?target .
        }
    }
    GROUP BY ?ttl_id
"""

interactions = dict()
for match in wp111g.query(int_q):
    result_dict = {key: str(val) for key, val in match.asdict().items()}
    interaction_types = {int_type.split("#")[1] for int_type in result_dict["types"].split(",")}
    result_dict["participants"] = set(result_dict["participants"].split(","))
    
    # don't need "Interaction"
    interaction_types.remove("Interaction")
    
    # Binding always with ComplexBinding so keep only ComplexBinding
    if "ComplexBinding" in interaction_types:
        result_dict["types"] = {"ComplexBinding"}
        
    else:result_dict["types"]  = interaction_types
    
    ttl_id = result_dict.pop("ttl_id")
    interactions[ttl_id] = result_dict

result_dict

{'types': {'Catalysis', 'DirectedInteraction'},
 'source': 'http://rdf.wikipathways.org/Pathway/WP111_r117097/Complex/ad4b9',
 'target': 'http://rdf.wikipathways.org/Pathway/WP111_r117097/WP/Interaction/c4d38',
 'participants': {'http://rdf.wikipathways.org/Pathway/WP111_r117097/Complex/ad4b9',
  'http://rdf.wikipathways.org/Pathway/WP111_r117097/WP/Interaction/c4d38'}}

## Check Nodes in Parsed Interactions

In [9]:
for iid, metadata in interactions.items():
    for node_id in metadata["participants"]:
        if node_id not in nodes:
            print(node_id)

http://rdf.wikipathways.org/Pathway/WP111_r117097/WP/Interaction/fee15
http://rdf.wikipathways.org/Pathway/WP111_r117097/WP/Interaction/c243a
http://rdf.wikipathways.org/Pathway/WP111_r117097/WP/Interaction/f52f4
http://rdf.wikipathways.org/Pathway/WP111_r117097/WP/Interaction/id86f308da
http://rdf.wikipathways.org/Pathway/WP111_r117097/WP/Interaction/c4d38
